Reconstruction of the crystal with ICSD code 60966. Notably, this crystal has the space group P 4 3 2, of which it is the only one (in my dataset at least). In the model used, this space group (/crystal) was not in the training set, and so the model has not been trained on it. This notebook shows it predicting the diffraction pattern.

In [1]:
import os
import sys
import torch
import cv2
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
from torchvision import transforms

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
from src.models.autoencoder import AELitModule

data_dir = "../data/FDP/"
sample_size = 1
ICSD_codes = os.listdir(data_dir)
chosen_codes = np.random.choice(ICSD_codes, sample_size)

In [1]:
run = "logs/train/40_C128_M128"

epoch = 199
# epochs_fakes = []
code = 198269

transform = transforms.ToTensor()
structures = torch.stack(
    [
        transform(
            np.load(
                os.path.join(data_dir, ICSD_code, ICSD_code + "_structure.npy")
            ).astype(np.float32)
        )
        for ICSD_code in [str(code)]
    ]
)
patterns = torch.stack(
    [
        transform(
            np.clip(
                np.load(os.path.join(data_dir, ICSD_code, ICSD_code + "_+0+0+0.npy")),
                0,
                1,
            ).astype(np.float32)
        )
        for ICSD_code in [str(code)]
    ]
)

epoch = str(epoch).zfill(3)
litmodule = AELitModule.load_from_checkpoint(
    os.path.join(os.getcwd(), "..", run, "checkpoints", f"epoch_{epoch}.ckpt")
)
litmodule.eval()
fakes = litmodule.model(structures.cuda())
fakes = fakes.view(sample_size * 128, 128).detach().cpu().numpy()
fake = 1 - fakes
cv2.imwrite("fake.png", 255 * fake)

NameError: name 'transforms' is not defined